Scaling and Clustering of the spotify library:

In [41]:
import scalingclustering
from scalingclustering import load_playlist_pkl, yellowbricking, apply_scaler, apply_kmeans, predict_kmeans, elbowing
import pandas as pd
import datetime


In [7]:
sp_library = load_playlist_pkl()

In [11]:
sp_library.head()

,playlistID,TrackName,TrackID,SampleURL,ReleaseYear,Genres,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms,Popularity
0,37i9dQZF1DWTvNyxOwkztu,Wonderful People - Original Mix,40sgR4OCBYNMpE3bQmlqJL,https://p.scdn.co/mp3-preview/5b7fa61078052357...,2020-02-21,[filter house],0.732,0.749,-9.133,0.0374,0.0422,0.902,0.102,0.349,121.016,7,0,467635,51.0
1,37i9dQZF1DWTvNyxOwkztu,Bubinga,1qh4v6N6Ozm9jHyuD9qBeB,https://p.scdn.co/mp3-preview/bf6e9d55788c40dd...,2019-08-23,[chill beats],0.689,0.82,-8.374,0.041,0.205,0.784,0.391,0.545,110.021,7,1,190203,50.0
2,37i9dQZF1DWTvNyxOwkztu,The Kármán Line,1dcu7ZenUoBB7J0r1sg5lz,https://p.scdn.co/mp3-preview/0ec7683528579c06...,2020-01-17,"[swiss indie, vapor soul]",0.819,0.512,-10.431,0.14,0.337,0.815,0.176,0.329,107.968,8,1,203657,50.0
3,37i9dQZF1DWTvNyxOwkztu,Alabora,3uI33EsE7AVM3GMxaOn7fv,https://p.scdn.co/mp3-preview/65374862914febbd...,2020-11-20,"[ethnotronica, organic electronic, organic house]",0.653,0.485,-14.745,0.0371,0.808,0.939,0.115,0.083,110.002,2,0,423756,55.0
4,37i9dQZF1DWTvNyxOwkztu,Kyoto,6Uk70qpuVpIbJYfpgp7FOv,https://p.scdn.co/mp3-preview/ff1896a57db36e8b...,2020-02-14,[unknown],0.436,0.614,-9.082,0.143,0.564,0.921,0.111,0.159,152.896,6,0,331359,52.0


In [20]:
pl_dict = {}
i = 0
for index in sp_library['playlistID'].value_counts().index:
    pl_dict[index] = i
    i += 1

In [21]:
pl_dict

{'37i9dQZF1DX6VdMW310YC7': 0,
 '37i9dQZF1DX0pH2SQMRXnC': 1,
 '37i9dQZF1DX0hvSv9Rf41p': 2,
 '37i9dQZF1DWTvNyxOwkztu': 3,
 '37i9dQZF1DX1OIMC8iDi74': 4,
 '37i9dQZF1DX3d2wagqzwmM': 5,
 '37i9dQZF1DX5xiztvBdlUf': 6,
 '37i9dQZF1DX5wDmLW735Yd': 7,
 '37i9dQZF1DX8ZMwsPgxIOs': 8,
 '37i9dQZF1DX3jWba5xiDhV': 9,
 '37i9dQZF1DX0BcQWzuB7ZO': 10,
 '37i9dQZF1DX8AliSIsGeKd': 11,
 '37i9dQZF1DX0AZ24QB6TCx': 12,
 '37i9dQZF1DX37cXruBmbRS': 13,
 '37i9dQZF1DX1tW4VlEfDSS': 14,
 '37i9dQZF1DX6J5NfMJS675': 15,
 '37i9dQZF1DX2pprEpa9URZ': 16,
 '37i9dQZF1DX1GT5IIzDqMe': 17,
 '37i9dQZF1DX11pUHJdXMnN': 18}

In [22]:
for pl_id in sp_library['playlistID']:
    sp_library['playlistnr'] = pl_dict[pl_id]
   

In [52]:
sp_library['Year'] = pd.to_datetime(sp_library['ReleaseYear'], format='%Y-%m-%d')
sp_library['Year'] = pd.DatetimeIndex(sp_library['Year']).year

In [53]:
sp_library.columns

Index(['playlistID', 'TrackName', 'TrackID', 'SampleURL', 'ReleaseYear',
       'Genres', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'key', 'mode', 'duration_ms', 'Popularity', 'playlistnr', 'Year'],
      dtype='object')

Columns for clustering:
'danceability'
'energy'
'loudness'
'speechiness'  # detects the presence of spoken words in a track
'acousticness'
'instrumentalness'
'liveness'  # Detects the presence of an audience in the recording
'valence'  # A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track
'tempo'


Add later when data exists:
'time_signature'
'popularity'

In [54]:
X = sp_library[['danceability',
                'energy',
                'loudness',
                'speechiness',
                'acousticness',
               'instrumentalness',
               'liveness', 
                'valence',  
                'tempo',
                'playlistnr',
                'Year',
                'Popularity']]

In [55]:
X

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlistnr,Year,Popularity
0,0.732,0.749,-9.133,0.0374,0.0422,0.902,0.102,0.349,121.016,8,2020,51.0
1,0.689,0.82,-8.374,0.041,0.205,0.784,0.391,0.545,110.021,8,2019,50.0
2,0.819,0.512,-10.431,0.14,0.337,0.815,0.176,0.329,107.968,8,2020,50.0
3,0.653,0.485,-14.745,0.0371,0.808,0.939,0.115,0.083,110.002,8,2020,55.0
4,0.436,0.614,-9.082,0.143,0.564,0.921,0.111,0.159,152.896,8,2020,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.757,0.938,-5.749,0.0642,0.000027,0.936,0.0733,0.125,144.996,8,2021,29.0
73,0.666,0.964,-7.804,0.0529,0.000384,0.815,0.101,0.806,134.987,8,2021,28.0
74,0.803,0.692,-11.32,0.0678,0.0268,0.773,0.157,0.495,126.007,8,2021,29.0
75,0.7,0.862,-8.434,0.0553,0.000114,0.939,0.0346,0.456,139.993,8,2021,24.0


In [56]:
X_scaled_df = apply_scaler(X)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlistnr,Year,Popularity
0,0.732,0.749,-9.133,0.0374,0.0422,0.902,0.102,0.349,121.016,8,2020,51.0
1,0.689,0.82,-8.374,0.041,0.205,0.784,0.391,0.545,110.021,8,2019,50.0
2,0.819,0.512,-10.431,0.14,0.337,0.815,0.176,0.329,107.968,8,2020,50.0
3,0.653,0.485,-14.745,0.0371,0.808,0.939,0.115,0.083,110.002,8,2020,55.0
4,0.436,0.614,-9.082,0.143,0.564,0.921,0.111,0.159,152.896,8,2020,52.0


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlistnr,Year,Popularity
0,0.640642,-0.025210,-0.428933,-0.567733,-0.395025,1.072749,-0.501106,0.101970,-0.327298,0.0,0.089251,0.640222
1,0.333471,0.384822,-0.218063,-0.514534,0.450260,0.765776,1.393781,0.912356,-0.856675,0.0,-0.083397,0.579021
2,1.262127,-1.393907,-0.789551,0.948434,1.135627,0.846421,-0.015910,0.019277,-0.955521,0.0,0.089251,0.579021
3,0.076305,-1.549834,-1.988090,-0.572166,3.581141,1.169004,-0.415869,-0.997840,-0.857590,0.0,0.089251,0.885024
4,-1.473835,-0.804847,-0.414764,0.992767,2.314250,1.122177,-0.442096,-0.683609,1.207631,0.0,0.089251,0.701422


In [57]:
n=3
kmeans = apply_kmeans(X_scaled_df, n)
kmeans.labels_

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
labels, clusters = predict_kmeans(X_scaled_df)

In [ ]:
clusters

In [ ]:
labels

In [ ]:
#clusters
pd.Series(labels).value_counts().sort_index()

In [ ]:
#X_df = pd.DataFrame(X)
X["label"] = labels
X

In [ ]:
X[X['label'] == 1].sample()

In [ ]:
elbowing(X_scaled_df)

In [ ]:
yellowbricking(X_scaled_df, n)